## Lab 6: Sequence Characteristics

##### Q0: What's your name?

The purpose of this exercise is to retrieve your gene's mRNA and protein sequences from Genbank and perform some (more) preliminary analysis.

Check that the fasta files containing the 5'UTR, CDS, and 3'UTR sequences of your gene (the files generated in lab exercise 4) are located in your working directory.

### 1. Identifying the GC content of regions of your mRNA.

In [3]:
from Bio import Entrez # we'll import a set of code written for querying NCBI databases using Entrez
from Bio import SeqIO # and a set of code specific for dealing with sequences

In [48]:
GI_id = "" # enter the accession # or GeneID for the mRNA sequence of your gene ***as a string***

Documentation for efecth http://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch  

Code adapted from Biopython Tutorial, 5.3.1  Parsing GenBank records from the net

In [2]:
Entrez.email = "" # let NCBI know who you are
handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id=GI_id) # db = database to query, rettype = retrieval type, retmode = retrieval mode 
nt_seq_record = SeqIO.read(handle, "gb") # read the contents "fetched" from NCBI
handle.close()

In [4]:
ntid = nt_seq_record.id # assign the accession number of the sequence (stored in the id attribute of nt_seq_record) to the variable ntid

#print the accession number
print(ntid)

ntseq = nt_seq_record.seq # assign the sequence (stored in the seq attribute of nt_seq_record) to the variable ntseq 

#print the sequence
print(ntseq)

#print the number of nucleotides in the sequence 
print("length:", len(ntseq))

Next we'll count the number of times each nucletide occurs in the sequence. In this lab exercise we will be counting the number of nucleotides over the entire mRNA sequence as well as within the 5'UTR, the CDS, and the 3'UTR. Instead of writing four for loops (one for each sequence/sub-sequence) it is much more efficient to write one function which you can call again and again. 

Funtions consist of a function definition, "def". The entire body of the definintion is indented. 

See the example function below.

In [16]:
def greetings(input_string): # this the defintion of a function called greetings, it takes as an argument a string called input_string
    output_string = input_string + " World" # the varibable output_string is the sum of input_string and the string " World"
    return output_string # returns output_string

input_string = "Hello" # assign input_string the value "Hello"
result = greetings(input_string) # the variable result is assigned to the result of sending the argument input_string into the function greetings 


print("input_string:", input_string)
print("result:", result)
print(greetings("UMB"))

Take the following code (written in a previous lab section) and turn it into a function called GC_content which takes a nucleotide sequence seq and returns a list called containing the counts of A's, T's, C's, and G's. 

In [6]:
def GC_content(sequence): # this is the definition of the function GC_content, it input consisting of a nucleotide sequence called sequence
    countA=0 # create variables to hold the count of each nucleotide
    countC=0
    countG=0
    countT=0
    countN=0 # create a variable to hold the count of any non-standard nucleotides
    
    for nt in sequence:#Loop over each nucleotide in the sequence
        if nt == "C": # ask if the nt is cytosine 
            countC +=1 # increase countC by one
        elif nt == "A": # ask if the nt is adenosine
            countA +=1 # increase countA by one
        elif nt == "T": # ask if the nt is thymine
            countT +=1 # increase countT by one
        elif nt == "G": # ask if the nt is guanine
            countG +=1 # increase countG by one
        else: # if above not True then nt is not a standard nucleotide
            countN +=1 # increase countN by one
            print("Non-standard nt", nt) # print a warning message

    #print("countA", countA) # print the values of the count variables
    #print("countC", countC)
    #print("countG", countG)
    #print("countT", countT)
    #print("countN", countN)
    
    counts = [countA, countT, countC, countG] # generate a list containing the number of A, T, C, and G in the sequence.

    return counts # return the list counts

ntseq = nt_seq_record.seq # assign the value of the variable ntseq to the variable sequence
# the variable counts is assigned to the result of sending the argument sequence into the function GC_content

counts = GC_content(ntseq)
print("nucleotide content = [A,T,C,G] ", GC_content(ntseq))

Q1: Explain the idea behind "reusable code". Why is this so important? Name at least two means by which you can make your code more reusable.

Next, calculate the number of cytosine and guanine nucleotides as a percentage of the total number of nt's in the sequence

In [17]:
assert len(counts) == 4 
#create a variable countGC to hold the count of G's and C's in the sequence

#define a variable, percGC, which is the countGC/(total number of nt)*100

#print the value of percGC
print("percGC =", percGC)

Q2: What does assert do and why is it useful?

Generate a list containing the number of A, T, C, G as a proportion of the total number of nucleotides.

In [18]:
# generate a list called propor to hold the proportions

# for each element x in counts
   
    # define a variable, p, which is the count/total number of nt
    
    # add a the calculated proportion to the list propor

    
# print the list propor
print("propors = ", propors)

Now we can create a graph - with the four nucleotides on the x-axis and the percentages on the y-axis.

In [19]:
import matplotlib.pyplot as plt # import a set of code written for generating graphics and tell the computer how to display graphs
%pylab inline 

nucleotides = arange(4) # the x-axis has four positions, one for each of the four nucleotides
bar(nucleotides, propor) # chart will be a bar chart, containing nucleotides on the x-axis and count as percent of length of the sequence on the y-axis
xticks(nucleotides + 0.5,  ('A', 'T', 'C', 'G') ) # label the x-axis with the nucleotides in the same order as in the counts list
title("Nucleotide composition of " + ntid) # add a title to the graph
ylabel("Proportion") # label the y-axis
show()

Next test if the nucleotide composition of your gene differs significantly from the expected equal representation of nucleotides using the chi-square test. Remember that the chi-square test statistic is the sum over all values (here each of the four nucleotides) of the difference between the number of times that nucleotide occurs within your sequence ("the observed") and the number of times you would expect to see that nucleotide if the nucleotides occur with equal frequencies ("the expected"), squared, divided by the expected. The smaller the difference between the observed and the expected, the smaller the test statistic will be and the less support you have for rejecting the null hypothesis that the observed and expected data come from the same distribution, i.e. are equally distributed. Assuming that the null hypothesis is true, the p-value represents the probability of finding a test statistic equal to or greater than your test statistic by chance. A p-value of 0.05 or less is the generally accepted standard for significance in scientific publications. Where the p-value is less than 0.05 we reject the null hypothesis; if the p-value is greater than 0.05 we fail to reject the null hypothesis. An excellent video reviewing the concept can be found here - http://youtu.be/HwD7ekD5l0g (ignore the part using the statistical software "R").

In [26]:
from scipy import stats # import a set of code written for computing statistical tests

def chi_calcs(sequence, counts): # define a function named chi_calcs for performing a chi-squared test on the nucleotide frequency within a sequence
    exp_counts = [int(round(0.25*len(sequence),0)), int(round(0.25*len(sequence),0)), int(round(0.25*len(sequence),0)), int(round(0.25*len(sequence),0))] # the expected number of each nucleotide in a sequence the length of your sequence at equal frequency 
    print("Expected number of nucleotides: ", exp_counts) # print out the list containing the expected frequency of each nucleotide
    obs_counts = counts # assign the observed frequency of each nucleotide in the list counts to the variable named obsfreq
    print("Observed number of nucleotides: ", obs_counts) # print out the list containing the observed frequency of each nucleotide
    assert len(exp_counts) == len(obs_counts) # assert that the number of items in the two lists are the equal

    chi2, pval = stats.chisquare(obs_counts, exp_counts) # calculates the chi2 test statistic (chi2) and the p-value (pval) using the observed and expected frequency lists
    print("Chi-squared test statistic: ", chi2) # print the value of the chi-square test statistic
    #print "p-value: ", pval # print the p-value
    return pval

# assign the value of the variable ntseq to the variable sequence
ntseq = nt_seq_record.seq

# send the arguments sequence and observed frequencies of nucleotides to the function chi_calcs
pval = chi_calcs(ntseq,counts)

print("p-value: ", pval)

#### Q3: Considering your mRNA sequence as a whole, are the four nucleotides present in equal frequencies?

As we've seen, the nucleotide content of a gene can differ significantly over the length of an mRNA sequence. Using the fasta files you generated last week, calculate the nucleotide content over the 5'UTR, CDS, and 3'UTR regions of your gene.

We'll use Biopython's SeqIO module to parse the fasta files into their header's and sequence's.


In [27]:
from Bio import SeqIO
GI_id = "NM_000321"
five_UTR_filename = GI_id + "_5UTR_seq.fasta" # the file where you saved your gene's 5'UTR sequence
CDS_filename = GI_id + "_CDS_seq.fasta" # the file where you saved your gene's CDS
three_UTR_filename = GI_id + "_3UTR_seq.fasta" # the file where you saved your gene's 3'UTR sequence

gene_regions = {} # create a dictionary called gene_regions

fasta_list = [five_UTR_filename, CDS_filename, three_UTR_filename] # put the file names into a list
for fl in fasta_list: # for each file, fl, in the list, fasta_list
    for s in SeqIO.parse(fl, "fasta"): # for each fasta record, s, in the file, fl, parse the fasta records (separates the text into the fasta header and sequence)
        print(s.id) # print the fasta header of the fasta record
        print(s.seq) # print the sequence contained in the fasta record
        
        # convert the sequence to a string
        
        
        # add an element to the dictionary, gene_regions, where the key is the fasta header and the value is the string of the sequence
         
        # print the length of the fasta sequence
        print(len(ntseq))  
        # print a newline to create a visual separation between the sequences
        print("\n")

For each sequence in the dictionary gene_regions, call the GC_content function. Remember that GC_content returns a list of the counts of each nucleotide in the sequence. Call the chi_calcs function, giving it the sequence and the list of counts - this will print out the observed and expected number of nucleotides, the chi-squared test statistic, and corresponding p-value.

In [5]:
# for each element of the dictionary gene_regions
for gene_region_key in gene_regions: 

    # print the dictionary key, the sequence id
    print(gene_region_key)
    
    # assign the nucleotide seq to a variable called sequence
    
    
    # send the sequence to the function GC_content, returns a list called counts containing the counts of each nucleotide  
    counts = GC_content(sequence) 
    
    # send the sequence and list counts to the function chi_calcs, this will print out the expected and observed nucletides, the chi-square statistic and the p-value
    pval = chi_calcs(sequence, counts) 
    print("pval =", pval)
    # print a newline as a visual separation between records
    print("\n")

#### Q4: Can you reject the null hypothesis of equal distribution of nucleotides for any of the regions of your gene? Explain. If you reject the null hypothesis, which nucletides appear to be over or under represented?

Now, let's generate a graph showing the percent GC of 50 nt bins accross your gene's entire mRNA sequence.

In [7]:
# create an empty list called region
regions = []

for i in range(0, len(ntseq)): # for each number i in the list of numbers from 1 to length of ntseq in steps of 50
    # set the variable gcnum to 0 
    
    for nt in ntseq[i:i+50]: # loop over the slice of nt_seq
        # ask if the nt is either G or C
        
            # if so increase the value of gcnum by 1
           
            
    # when the loop over the slice is complete, calculate the percent of nts in the region that are G or C
    
    #add the percentage to the list "region"
    
    
print(regions)

We'd like to plot the GC content by region over the length with the coding region of your gene highlighted. To do this we first need to know the begining and ending position of the coding region. Write a for loop using the features attribute of nt_seq_record; if the feature type is "CDS", assign the starting position to cds_start and the ending position to cds_end.

In [8]:
# for each feature in this record's features, using the .features attribute of nt_seq_record
for f in nt_seq_record.features: 
    
    # ask if the feature is a CDS, using the .type attribute of the feature
    if (f.type == "CDS"):
        # if so, assign the start location to the variable cds_start, using the .location.start attribute of the feature
        
        # if so, assign the end location to the variable cds_end, using the .location.end attribute of the feature
        
        
# print the value of cds_start
print(cds_start)

# print the value of cds_end
print(cds_end)

In [9]:
import matplotlib.pyplot as plt # import a set of code written for generating graphics and tell the computer how to display graphs
%pylab inline

plt.plot(region) # plot the values of region
plt.axhline(0, color='black') # forces the y-axis to start at 0
plt.axhline(50, color='red') # add a red line representing the average GC content of the entire sequence

plt.axvspan(cds_start, cds_end, facecolor='grey', alpha=0.5) # create a grey region corresponding to the coding region
plt.ylabel("Percent") # label the y-axis
plt.xlabel("nt position (5'-3')") # label the x-axis
plt.title("GC percentage of " + ntid + "\n calculated over 50 nt regions") # add a title
plt.show()

#### Q5: Are there noticeable differences in the GC content between the different regions of the mRNA sequence? Explain.

That's all folks!  
Please save your notebook and upload the notebook to Blackboard.